# DETECÇÃO DOS SINAIS

* Importação das bibliotecas necessárias para rodar o script

In [1]:
import pickle
import cv2
import mediapipe as mp
import numpy as np
import time

* Importa o modelo gerado pelo script "Treino_Classificacao"

In [3]:
model_dict = pickle.load(open('./model_md7.p', 'rb'))
model = model_dict['model']

* Realiza a inicialização da câmera

In [ ]:
cap = cv2.VideoCapture(0)

* A variável mp_hands é responsável por fornecer as funcionalidades de rastreamento de mãos. Com ela, você pode detectar e acompanhar as posições das mãos em uma imagem ou vídeo.</br>
* A variável mp_drawing fornece funções utilitárias para desenhar os landmarks, conexões e outras anotações sobre a imagem de saída. Isso é útil para visualizar os resultados do rastreamento das mãos.</br>
* Por fim, a variável mp_drawing_styles fornece estilos de desenho predefinidos para os landmarks e conexões identificados nas mãos rastreadas.</br>

In [ ]:
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

* Cria um objeto hands que será usado para rastrear mãos em uma imagem estática, com uma confiança mínima de detecção definida como 0.3 (30%).

In [ ]:
hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.3)

* Dicionário para fazer a transformação das labels para o texto correto que vai aparecer na câmera quando o simbolo for reconhecido.

In [ ]:
labels_dict = {1: '1', 2: '2', 4: '4', 5:'5', 7:'7', 9:'9',
    'A':'A','AVIAO': 'AVIAO', 'B':'B', 'C':'C', 'D':'D', 'E':'E', 'F':'F', 'G':'G',
    'I':'I', 'L':'L', 'M':'M', 'N':'N', 'O':'O', 'P':'P', 'PALAVRA':'PALAVRA', 'PEQUENO':'PEQUENO',
    'Q':'Q', 'R':'R', 'S':'S', 'T':'T', 'U':'U', 'V':'V', 'W':'W', 'X':'X','Y':'Y'}

* Detecção e  rastreamento em tempo real das mãos em um vídeo capturado pela câmera, bem como a previsão de um caractere com base nas informações extraídas dos landmarks das mãos usando o modelo treinado. O resultado é exibido no quadro de vídeo em tempo real.

In [ ]:
# Inicialização do OpenCV e do MediaPipe
cap = cv2.VideoCapture(0)
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# Inicialização das variáveis
data_aux = []
x_ = []
y_ = []
fps_start_time = time.time()
fps_counter = 0

while True:
    ret, frame = cap.read()

    H, W, _ = frame.shape

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    results = hands.process(frame_rgb)

    if results.multi_hand_landmarks:
        if len(results.multi_hand_landmarks) == 1:
            hand_landmarks = results.multi_hand_landmarks[0]

            mp_drawing.draw_landmarks(
                frame,
                hand_landmarks,
                mp_hands.HAND_CONNECTIONS,
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style())

            for i in range(len(hand_landmarks.landmark)):
                x = hand_landmarks.landmark[i].x
                y = hand_landmarks.landmark[i].y

                x_.append(x)
                y_.append(y)

                data_aux.append(x - min(x_))
                data_aux.append(y - min(y_))

            x1 = int(min(x_) * W)
            y1 = int(min(y_) * H)

            x2 = int(max(x_) * W)
            y2 = int(max(y_) * H)

            prediction = model.predict([np.asarray(data_aux)])

            if isinstance(prediction[0], str):
                predicted_character = prediction[0]
            else:
                predicted_character = labels_dict.get(prediction[0], 'Desconhecido')

            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 0), 4)
            cv2.putText(frame, predicted_character, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (120, 120, 120), 3,
                        cv2.LINE_AA)

            x_ = []
            y_ = []
            data_aux = []

    # Cálculo do FPS
    fps_counter += 1
    if (time.time() - fps_start_time) > 0:
        fps = fps_counter / (time.time() - fps_start_time)
        fps_counter = 0
        fps_start_time = time.time()
        cv2.putText(frame, f'FPS: {round(fps, 2)}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)
    

    cv2.imshow('frame', frame)
    if cv2.waitKey(1) == 27:  # Press 'Esc' to exit
        break

cap.release()
cv2.destroyAllWindows()
